# Estimating the impact of Government measures on COVID-19 infection spread through mobility data

We aim at answering the question :

**How does is the implementation of different strategies affecting the rates of COVID-19 infection ?**

We build a bayesian framework, that leverages an epidemiological compartmental model, in order to estimate the real effective reproduction number $R_t$ through time as a function of mobility data.  
We also build another model that predicts mobility based on non-pharmaceutical interventions (NPIs), in order to build an end-to-end pipeline from NPIs to $R_t$ and compartments, e.g. number of hospitalized, critical and deceased individuals.   
This allows us to build _what if_ scenarios, useful to evaluate governement policies both restrospectively and prospectively.


![process](covid_estimation.svg)

## Measures dataset

Description of Oxford dataset, categories c1 to c8

## Mobility datasets
We gathered data from Google and Apple, describing respectively 6 and 3 mobility categories.

Google
- _Retail and recreation_
    - restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.
- _Grocery and pharmacy_
    - grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies.
- _Parks_
    - national parks, public beaches, marinas, dog parks, plazas,and public gardens
- _Transit stations_
    - public transport hubs such as subway, bus, and train stations
- _Workplaces_
- _Residential_

_The baseline is the median value, for the corresponding day of the week, during the 5-week period Jan 3–Feb 6, 2020._

Apple
- _Driving_
- _Walking_
- _Transit_

_relative volume of directions requests per country/region, sub-region or city compared to a baseline volume on 13 January 2020_

## SEIR-HCD Model

Model built on compartmental models. Each of the following letters represents a compartment of the population of a country:

S - susceptible  
E - exposed  
I - infectious  
R - recovered  
H - hospitalized  
C - critical  
D - deceased  

We normalize these count numbers by the total population of the country.

In [8]:
%%latex
\begin{align}
\frac{dS}{dt} &= - \frac{R_t}{T_{inf}} I S \\
\frac{dE}{dt} &= \frac{R_t}{T_{inf}} I S - \frac{E}{T_{inc}} \\
\frac{dI}{dt} &= \frac{E}{T_{inc}} - \frac{I}{T_{inf}} \\
\frac{dR}{dt} &= m_a \frac{I}{T_{inf}} + (1 - c_a)\frac{H}{T_{hosp}} \\
\frac{dH}{dt} &= (1 - m_a) \frac{I}{T_{inf}} + (1 - f_a)\frac{C}{T_{crit}} - \frac{H}{T_{hosp}} \\
\frac{dC}{dt} &= c_a \frac{H}{T_{hosp}} - \frac{C}{T_{crit}} \\
\frac{dD}{dt} &= f_a \frac{C}{T_{crit}}
\end{align}

<IPython.core.display.Latex object>

### Parameters used in the model
$R_t$ = reproduction number at time t.  
Typical 3.6* at t=0

Transition times

    T_inc = average incubation period. Typical 5.6* days
    T_inf = average infectious period. Typical 2.9 days
    T_hosp = average time a patient is in hospital before either recovering or becoming critical. Typical 4 days
    T_crit = average time a patient is in a critical state (either recover or die). Typical 14 days

Fractions  
These constants are likely to be age specific (hence the subscript a):

    m_a = fraction of infections that are asymptomatic or mild. Assumed 80% (i.e. 20% severe)
    c_a = fraction of severe cases that turn critical. Assumed 10%
    f_a = fraction of critical cases that are fatal. Assumed 30%
    
*Averages taken from https://www.kaggle.com/covid-19-contributions

#### Priors for these parameters
- The Conversation [article](https://theconversation.com/how-long-are-you-infectious-when-you-have-coronavirus-135295):
    - Incubation: 5 days
    - Infectious period: 2 days in incubation, 6 days from symptoms onwards
    - Illness: 10 days
    - NB: _For COVID-19, there is emerging evidence to suggest the infectious period may start 1 to 3 days before you develop symptoms_ 
        - We can reduce incubation time by 2 days and increase infectious time by 2
    - [article](https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30196-1/fulltext)
        - _The peak viral load of patients with MERS-CoV and SARS-CoV infections occurs at around 7–10 days after symptom onset_ 
        - _The median interval between symptom onset and hospitalisation was 4 days (range 0–13)_

- Imperial
    - T_inc: Gamma(5.1, 0.86)
        - _The infection-to-onset distribution is Gamma distributed with mean 5.1 days and coefficient of variation 0.86._   
    - T_inf + T_hosp + T_crit: Gamma(17.8, 0.45)
        - _The onset-to-death distribution is also Gamma distributed with a mean of 17.8 days and a coefficient of variation 0.45._  
    - T_inc + T_inf + T_hosp + T_crit
        - _The infection-to-death distribution is therefore given by: π∼Gamma(5.1,0.86) + Gamma(17.8,0.45)_

### Reproduction number
let $\mathrm{m}_{t, i}$ be the reduction of mobility in the category $i$, relatively to a baseline (before the pandemic).  
Hence $\mathrm{m}_{t, i} > -1$.

We adopt the strong hypothesis that each mobility category presents the same transmission dynamics. Let $R_{t, i}$ be defined by the following linear relationship:
$$ R_{t, i} = (1 + \mathrm{m}_{t, i})R_0 - \mathrm{m}_{t, i}R_1  $$
for any time $t$ through the pandemic, with $R_0$, $R_1$ to be estimated.

We model the effective reproduction number with a weighted mean for each mobility category: 

$$R_t = \sum_i \alpha_i R_{t, i}$$
with $\alpha_i$ to be estimated, where
- $\sum_i \alpha_i = 1$ 
- $\forall i, \alpha_i > 0$

We choose the following prior : $\forall i, \alpha_i \sim \mathrm{Gamma}(1, .5)$

## Bayesian modelling

We use the numpyro probabilistic programming framework.

We use the following parameterizations:
- (concentration, rate) for the $\mathrm{GammaPoisson}$ distribution.
- (mean, std) for the $\mathrm{Gamma}$ distribution
- (mean, sample size) for the $\mathrm{Beta}$ distribution

### Target
Let $D_t$ be the number of death at time $t$ for a given country. We model $d_t = \mathbb{E}[D_t]$

We sample 
$$D_t \sim \mathrm{GammaPoisson}(\psi, \frac{\psi}{d_t})$$
where $\psi \sim \mathcal{N}^+(0, 5)$

### Compartment init
We seed the model with the following, assuming $N$ is the country population:
- $I_0 \sim \mathrm{Gamma}(100, 50) / N$
- $S_0 = 1 - I_0$
- $E_0, R_0, H_0, C_0, D_0 = 0$

### Other priors
$R_0 \sim \mathcal{N}^+(3.28, \kappa_0)$

$R_1 \sim \mathcal{N}^+(0.7, \kappa_1)$

where $\kappa_0, \kappa_1 \sim \mathcal{N}^+(0, .5)$


$T_{inc} \sim \mathrm{Gamma}(5.6, .86)$

$T_{inf} \sim \mathrm{Gamma}(2.9, 1)$

$T_{hosp} \sim \mathrm{Gamma}(4, 1)$

$T_{crit} \sim \mathrm{Gamma}(14, 1)$

$m_a \sim \mathrm{Beta}(0.8, \phi_m)$

$c_a \sim \mathrm{Beta}(0.1, \phi_c)$

$f_a \sim \mathrm{Beta}(0.35, \phi_f)$

where $\phi_m, \phi_c, \phi_f \sim \mathrm{Gamma}(7, 2)$


# Relationship between measures and mobility

## Action plan
We can leverage ACAPS and/or Oxford datasets. 

### Approach 1: bayesian regression
- Use events data like Imperial, e.g. $m_t = \exp(- \sum \alpha_k I_{k, t}) - 1$ with $I_{k, t} \in \{0, 1\}$ indicator of the measure $k$, and $\alpha_k$ to be inferred
- Same thing but with $I_k \geq 0$ quantifying the impact (Oxford dataset)
- Add lag features ? Exponentially weighted mean of indicators ?
- Regularization : Ridge, Lasso ?
- Fit model on one country vs multiple country
    - how to handle country-specific behaviors e.g. parks in Sweden, Denmark ? Is it / should it be implemented in the measure indicator ?

### Approach 2: infer causality ?

- [Article](https://towardsdatascience.com/inferring-causality-in-time-series-data-b8b75fe52c46#99db): _Inferring causality in time series data_
    - [Tigramite](https://github.com/jakobrunge/tigramite): PCMCI - _Causal discovery for time series datasets_
- Paper [Graphical models for time series](https://sci-hub.im/https://ieeexplore.ieee.org/document/5563116): Gaining insight into their computational implementation

# Combining to make scenarios

- What if the lockdown was decided one week later ? Two weeks earlier ?
- What will happen if the lockdown is lifted in one week ? Will we see a spike in hospitals ? #flatten the curve


- Input : measure, date of implementation, country
- Output : deaths [+ infected, hospitalized, critical] time series

# Future work

- We could leverage hospitalization and ICU previsions from IHME and feed it to the model, so that it learns based on observations of H, C and D compartments at once. This would reinforce the confidence one can have on the deceased predictions, as well as enable the model to estimate other compartments more accurately.
- We could include Citymapper [mobility index](https://citymapper.com/cmi/) as an additional mobility measurement, as they do [there](https://www.medrxiv.org/content/10.1101/2020.04.05.20054288v2)

# Sources


[Prior work](https://github.com/horaceg/uncover/blob/master/working/Measures%20and%20rate.html) (to be downloaded):  
- Map + time series of measures with cases and mobility
- Minimization of parameters for the compartment model

*Sources*

- Imperial team
    - [Report 13](https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-13-europe-npi-impact/): 30 March 2020
    - [Technical update](https://arxiv.org/abs/2004.11342): 23 april 2020
    - [Code](https://github.com/ImperialCollegeLondon/covid19model) in R and Stan
    - [website](https://mrc-ide.github.io/covid19estimates/#/)


- R0: [An article estimating it](https://academic.oup.com/jtm/article/27/2/taaa021/5735319) 13 Feb 2020


- Compartment models
    - [Wikipedia](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology)
    - SEIR-HCD [Kaggle kernel](https://www.kaggle.com/anjum48/seir-hcd-model) from @anjum (Datasaurus). Sources of inspiration:
        - [Neher labs](https://covid19-scenarios.org/) 
        - [Gabriel Goh](https://gabgoh.github.io/COVID/index.html) 


- Probabilistic programming
    - [numpyro](https://github.com/pyro-ppl/numpyro) ([paper](https://arxiv.org/abs/1912.11554)): probabilistic programming library in python, built on [JAX](https://github.com/google/jax)
    - [SEIR ode modelling](https://docs.pymc.io/notebooks/ODE_API_introduction.html) in Pymc3 
    - [Predator-Prey ode](http://pyro.ai/numpyro/examples/ode.html) in Numpyro 
    - Another [predator-prey](https://fehiepsi.github.io/rethinking-numpyro/16-generalized-linear-madness.html) ode in numpyro
    - [A bayesian SIR](https://github.com/twiecki/covid19) modelling with Pymc3 with the [video](https://youtu.be/C1kWBTj6KvE?t=410)
    - [Estimating COVID-19's $R_t$ in Real-Time with PYMC3](https://github.com/k-sys/covid-19/blob/master/Realtime%20Rt%20mcmc.ipynb) by Keving Systrom, founder of Instagram


- Other submissions
    - previous winning solution : [Worldwide measures implementation](https://www.kaggle.com/agneshui/worldwide-measures-implementations)
    - [Covid19 reopening impact with r estimation](https://www.kaggle.com/wjholst/covid-19-reopening-impact-with-r-estimation)


- Misc
    - [generation interval](https://www.medrxiv.org/content/10.1101/2020.03.05.20031815v1)
    - [masks](https://www.preprints.org/manuscript/202004.0203/v1)

*Data sources*

- Epidemiology
    - [Our world in data](https://github.com/owid/covid-19-data/tree/master/public/data): confirmed cases and deaths from ECDC (European center of Disease Control) + tests (collected by owid)


- Mobility
    - [Apple](https://www.apple.com/covid19/mobility) - 3 categories: walking, driving, transit
    - [Google](https://www.google.com/covid19/mobility/) - 6 categories


- Government Measures
    - [ACAPS](https://data.humdata.org/dataset/acaps-covid19-government-measures-dataset)
    - [Oxford](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker)
